# This notebook will be building, preparing, training, and downloading a CNN (Convolutional Neural Network) from scratch, it is using only numpy

### Importing the dependencies

In [ ]:
import numpy as np
from keras.datasets import mnist
import pickle
import sys
import os
sys.path.append(os.path.abspath('..'))
from layers import Activation, Sigmoid, Tanh, ReLU, Softmax, Convolutional, Dense, Layer, Reshape, categorical_cross_entropy_prime, categorical_cross_entropy

### Defining the training functions

In [ ]:
def train(network, loss, loss_prime, x_train, y_train, epochs, learning_rate):
    for epoch in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # Forward pass
            output = x
            for layer in network:
                output = layer.forward(output)
            # Compute loss
            error += loss(y, output)
            # Backward pass
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)
        error /= len(x_train)
        print(f'Epoch {epoch+1}/{epochs}, Error={error}')

def predict(network, x):
    output = x
    for layer in network:
        output = layer.forward(output)
    return output

### Data Preprocessing

In [ ]:
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Preprocess data

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape input data

x_train = x_train.reshape(-1, 1, 28, 28)
x_test = x_test.reshape(-1, 1, 28, 28)

# One-hot encode labels

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Reshape labels to match output shape

y_train = y_train.reshape(-1, 10, 1)
y_test = y_test.reshape(-1, 10, 1)

# Limit the training data for faster training

train_limit = 1000  # Adjust as needed
test_limit = 200    # Adjust as needed

x_train = x_train[:train_limit]
y_train = y_train[:train_limit]

x_test = x_test[:test_limit]
y_test = y_test[:test_limit]

### Building the CNN

In [ ]:
network = [
    Convolutional((1, 28, 28), kernel_size=3, depth=8),
    ReLU(),
    Reshape((8, 26, 26), (8 * 26 * 26, 1)),
    Dense(8 * 26 * 26, 128),
    ReLU(),
    Dense(128, 10),
    Softmax()
]

### Training

In [ ]:
epochs = 170  
learning_rate = 0.006 

train(network, categorical_cross_entropy, categorical_cross_entropy_prime, x_train, y_train, epochs, learning_rate)

### Testing

In [ ]:
correct = 0
total = len(x_test)

for x, y in zip(x_test, y_test):
    output = predict(network, x)
    if np.argmax(output) == np.argmax(y):
        correct += 1

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100}%')

### Saving the model

In [ ]:
def save_model(network, filename):
    params = []
    for layer in network:
        layer_params = {}
        if hasattr(layer, 'weights'):
            layer_params['weights'] = layer.weights
        if hasattr(layer, 'biases'):
            layer_params['biases'] = layer.biases
        params.append(layer_params)
    with open(filename, 'wb') as f:
        pickle.dump(params, f)

save_model(network, 'cnn_mnist_model3.pkl')